In [ ]:
import numpy as np
from scipy.optimize import root
import plotly.graph_objects as go

# Course 3 : N-body problem

In [ ]:
class three_body_model:

    def __init__(self):
        self.m0 = 1.00000597682
        self.m1 = 9.54786104043e-4
        self.m2 = 2.85583733151e-4
        self.G = 2.95912208286e-4

    def fcn(self, t, y):
        q = y[0:9]
        p = y[9:18]
        q_dot = self.H_p(p)
        p_dot = -self.H_q(q)
        return np.concatenate((q_dot, p_dot))

    def H_p(self, p):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        Hp = np.zeros(9)
        Hp[0:3] = p[0:3]/m0
        Hp[3:6] = p[3:6]/m1
        Hp[6:9] = p[6:9]/m2
        return Hp
    
    def H_q(self, q):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        Hq = np.zeros(9)
        Hq[0:3] = ( G*m0*m1*((q[0:3]-q[3:6])/np.power(np.linalg.norm(q[0:3]-q[3:6]),3)) 
                   +G*m0*m2*((q[0:3]-q[6:9])/np.power(np.linalg.norm(q[0:3]-q[6:9]),3)))
        Hq[3:6] = ( G*m1*m0*((q[3:6]-q[0:3])/np.power(np.linalg.norm(q[3:6]-q[0:3]),3))
                   +G*m1*m2*((q[3:6]-q[6:9])/np.power(np.linalg.norm(q[3:6]-q[6:9]),3)))
        Hq[6:9] = ( G*m2*m0*((q[6:9]-q[0:3])/np.power(np.linalg.norm(q[6:9]-q[0:3]),3)) 
                   +G*m2*m1*((q[6:9]-q[3:6])/np.power(np.linalg.norm(q[6:9]-q[3:6]),3)))
        return Hq

    def hamiltonian(self, y):
    
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        nt = y.shape[1]
        neq = y.shape[0]
        q = y[0:neq//2]
        p = y[neq//2:neq]
        ham = np.zeros(nt)

        for i in range(nt):
            ham[i] = ( 0.5*( (1/m0)*np.dot(p[0:3,i],p[0:3,i]) 
                            +(1/m1)*np.dot(p[3:6,i],p[3:6,i]) 
                            +(1/m2)*np.dot(p[6:9,i],p[6:9,i]) ) 
                      - G *( (m0*m1)/np.linalg.norm(q[0:3,i]-q[3:6,i]) 
                            +(m0*m2)/np.linalg.norm(q[0:3,i]-q[6:9,i])
                            +(m1*m2)/np.linalg.norm(q[3:6,i]-q[6:9,i]) ) )

        return ham

## Integration with order 1 methods

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

def symplectic_euler(tini, tend, nt, yini, H_p, H_q):
    
    dt = (tend-tini) / (nt-1) 
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    q, p = np.split(y, 2)
    for it in range(nt-1):
        q_n = q[:,it]
        p_n = p[:,it]
        q[:,it+1] = q_n + dt*H_p(p_n)
        p[:,it+1] = p_n - dt*H_q(q[:,it+1])

    return ode_result(y,t)

In [ ]:
# initialization
m1 = 9.54786104043e-4
m2 = 2.85583733151e-4

qini = np.zeros(9)
qini[0] =  0.
qini[1] =  0.
qini[2] =  0.
qini[3] = -3.5023653
qini[4] = -3.8169847
qini[5] = -1.5507963
qini[6] =  9.0755314
qini[7] = -3.0458353
qini[8] = -1.6483708

pini = np.zeros(9)
pini[0] =  0.
pini[1] =  0.
pini[2] =  0.
pini[3] =  0.00565429*m1
pini[4] = -0.00412490*m1
pini[5] = -0.00190589*m1
pini[6] =  0.00168318*m2
pini[7] =  0.00483525*m2
pini[8] =  0.00192462*m2

yini = np.concatenate((qini, pini))

tini = 0.
tend = 11000.
nt = 1501

tbm = three_body_model()
H_p = tbm.H_p
H_q = tbm.H_q
fcn = tbm.fcn

# forward Euler integration
sol_fe = forward_euler(tini, tend, nt, yini, fcn)
yfe = sol_fe.y

# backward Euler integration
sol_be = backward_euler(tini, 0.95*tend, nt, yini, fcn)
ybe = sol_be.y

# symplectic Euler integration
sol_se = symplectic_euler(tini, tend, nt, yini, H_p, H_q)
yse = sol_se.y

fig = go.Figure()
fig.add_trace(go.Scatter(x=yfe[3]-yfe[0], y=yfe[4]-yfe[0], line_color='blue', legendgroup='fe', 
                         legendgrouptitle_text="Forward Euler", name='Jupiter'))
fig.add_trace(go.Scatter(x=yfe[6]-yfe[0], y=yfe[7]-yfe[0], line_color='blue', legendgroup='fe',
                         legendgrouptitle_text="Forward Euler", line_dash='dash', name='Saturn'))
fig.add_trace(go.Scatter(x=ybe[3]-ybe[0], y=ybe[4]-ybe[0], line_color='red', legendgroup='be', 
                         legendgrouptitle_text="Backward Euler", name='Jupiter'))
fig.add_trace(go.Scatter(x=ybe[6]-ybe[0], y=ybe[7]-ybe[0], line_color='red', legendgroup='be',
                         legendgrouptitle_text="Backward Euler", line_dash='dash', name='Saturn'))
fig.add_trace(go.Scatter(x=yse[3]-yse[0], y=yse[4]-yse[0], line_color='green', legendgroup='se', 
                         legendgrouptitle_text="Symplectic Euler", name='Jupiter'))
fig.add_trace(go.Scatter(x=yse[6]-yse[0], y=yse[7]-yse[0], line_color='green', legendgroup='se',
                         legendgrouptitle_text="Symplectic Euler", line_dash='dash', name='Saturn'))
fig.show()

## Hamiltonian

In [ ]:
ham_fe = tbm.hamiltonian(sol_fe.y)
ham_be = tbm.hamiltonian(sol_be.y)
ham_se = tbm.hamiltonian(sol_se.y)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol_fe.t, y=ham_fe, line_color='blue', name='Forward Euler'))
fig.add_trace(go.Scatter(x=sol_be.t, y=ham_be, line_color='red', name='Backward Euler'))
fig.add_trace(go.Scatter(x=sol_se.t, y=ham_se, line_color='green', name='Symplectic Euler'))
fig.update_yaxes(exponentformat='e')
fig.update_layout(title="Hamiltonian")
fig.show()